In [399]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [400]:
data=pd.read_csv('Full+data.csv')

In [401]:
extra=pd.read_csv('extra_data.csv')

In [402]:
attributes=pd.read_excel('Womens+Attributes.xlsx')

In [403]:
USC_product_attributes=pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx')

In [404]:
attributes=attributes[['Style',
       '(Reference Style Lookbook, choose all that apply)']]

In [405]:
USC_product_attributes.head()

,product_id,product_color_id,attribute_name,attribute_value
0,01DVBTBPHR8WJTCVEN5AJRHF47,01DVBTBPJ41VVT00JJCG8TTZ2W,gender,Women
1,01DVA7QRXM928ZM0WWR7HFNTC1,01DVA7QRXXR9F0TWVE1HMC5ZQ3,Primary Color,Blacks
2,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual
3,01E1JM43NQ3H17PB22EV3074NX,01E1JM5WFWWCCCH3JTTTCYQCEQ,style,Modern
4,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium


In [406]:
#Dropped all columns with ALL null values
data_dropped=data.dropna(axis=1,how='all')

In [407]:
#Attach additional data to full data
full_data=pd.concat([data_dropped,extra])

In [408]:
final_data=pd.merge(full_data,USC_product_attributes,on='product_id',how='left')

In [409]:
len(final_data)

87229

In [410]:
#No duplicate rows
len(final_data.drop_duplicates())

87229

In [411]:
#5655 rows have attribute occasion
#final_data[final_data['attribute_name']=='occasion']

In [412]:
#See how many occasion exists, These are the occasions we need to look for and find rules for
final_data[final_data['attribute_name']=='occasion']['attribute_value'].unique()

array(['Weekend', 'Work', 'Day to Night', 'Night Out', 'Vacation',
       'Workout'], dtype=object)

In [413]:
#Lets start with Night Out - What clothing items are tagged as night
night_out=final_data[final_data['attribute_value']=='Night Out']

In [414]:
night_out_subset=night_out[['brand','brand_category','description','details','product_id','name','tsv']]

In [415]:
# # % each word comes
# night_out_subset['brand'].value_counts(1)*100

In [416]:
final_data['Night_out_target']=np.where(final_data['attribute_value']=='Night Out',1,0)
    

In [417]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [418]:
subset_for_model=final_data[['brand_category','description','details','name','product_id','tsv','Night_out_target']]

In [419]:
#Remove / from brand_category by space
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace('/',' ')
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace(',',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace('\n',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace('/',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace(',',' ')

In [420]:
subset_for_model.head()
subset_for_model.fillna('Empty',inplace=True)

# Text preprocessing

In [421]:
#Remove stopwords
nltk.download('stopwords')
columns=['details','description','name','brand_category']
for j in columns:
    corpus = []
    for i in range(0, 10):
        new=re.sub('[^a-zA-Z]', ' ',str(subset_for_model.loc[i,j]))
        new = new.lower()
        new = new.split()
        ps = PorterStemmer()
        new = [ps.stem(word) for word in new if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        subset_for_model.loc[i,j]=new


    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Using a bag of words model 

In [422]:
subset_for_model.head()


,brand_category,description,details,name,product_id,tsv,Night_out_target
0,themensstor shoe sneaker lowtop,vintag fit leather sneaker logo print design s...,themensstor shoe sneaker lowtop,origin fit sneaker,01DSRPSZTDW2PGK1YWYXJGKZZ0,"'design':12 'fila':1A 'fit':3A,6 'leather':7 '...",0
1,unknown,empti,unknown,hat,01DSQXJBX0R7DCW7KTAC1SW547,'chanel':1A 'hat':2A,0
2,accessori,timeless leather belt craft smooth cowhid mini...,accessori,petit oval buckl belt,01DPGV8TGRAB993PF7Z3YWG2VR,"'belt':5A,9 'buckl':4A,21 'cowhid':13 'craft':...",0
3,justkid girl girl swimwearcoverup justkid girl...,pretti ruffl sleev trim elev essenti tank swim...,justkid girl girl swimwearcoverup justkid girl...,littl gir girl ariana one piec upf swimsuit,01DSR8G3F7DBRTMP8THF97XSQ2,'50':14A 'allov':28 'ariana':9A 'color':27 'el...,0
4,justkid babi month infantgirl footiesromp,versatil convert gown eleph appliqu,justkid babi month infantgirl footiesromp,babi girl endear eleph pima cotton convert gown,01DSR8G5GP519DEDCSKBMWQVK5,"'appliqu':17 'babi':3A 'convert':10A,13 'cotto...",0


In [486]:
subset_for_model['Mix_of_items']=subset_for_model['brand_category']+subset_for_model['description']+subset_for_model['name']+subset_for_model['details']

In [487]:
# for j in columns:
#     cv=CountVectorizer(max_features = 1500)
#     X=cv.fit_transform(subset_for_model).toarray()
#     y = subset_for_model.iloc[:,-1].values

In [488]:
data=subset_for_model['Mix_of_items']
y = subset_for_model.iloc[:,-1].values

In [489]:
import scipy.sparse as sp
vect=CountVectorizer(max_features=1500)
X=vect.fit_transform(data)


In [490]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

In [491]:
classifier = GaussianNB()
classifier.fit(X_train.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [492]:
y_pred_NB = classifier.predict(X_test.toarray())
y_pred_NB

KeyboardInterrupt: 

In [429]:
cm_NB = confusion_matrix(y_test, y_pred_NB) 
cm_NB

array([[29495,  5139],
       [   48,   210]])

In [430]:
#Accuracy
(29495+210)/34892*100

85.13412816691506

In [431]:
210/(210+48)

0.813953488372093

# Try on Full Data for occasions

In [432]:
occasion=final_data[final_data['attribute_name']=='occasion']

In [433]:
occasion['brand_category']=occasion['brand_category'].str.replace('/',' ')
occasion['brand_category']=occasion['brand_category'].str.replace(',',' ')
occasion['details']=occasion['brand_category'].str.replace('\n',' ')
occasion['details']=occasion['brand_category'].str.replace('/',' ')
occasion['details']=occasion['brand_category'].str.replace(',',' ')

In [434]:
occasion.head()
occasion.fillna('Empty',inplace=True)
occasion=occasion.reset_index()

In [435]:
#Remove stopwords
nltk.download('stopwords')
columns=['details','description','name','brand_category']
for j in columns:
    corpus = []
    for i in range(0, len(occasion)):
        new=re.sub('[^a-zA-Z]', ' ',str(occasion.loc[i,j]))
        new = new.lower()
        new = new.split()
        ps = PorterStemmer()
        new = [ps.stem(word) for word in new if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        occasion.loc[i,j]=new

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [436]:
fdata=occasion[['brand_category','description','details','name']]
y = occasion['attribute_value'].values

In [437]:
import scipy.sparse as sp
vect=CountVectorizer(max_features=1500)
X=sp.hstack(fdata.apply(lambda col: vect.fit_transform(col)))

In [438]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

In [439]:
classifier = GaussianNB()
classifier.fit(X_train.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [440]:
y_pred_NB = classifier.predict(X_test.toarray())
y_pred_NB

array(['Day to Night', 'Workout', 'Day to Night', ..., 'Weekend',
       'Day to Night', 'Vacation'], dtype='<U12')

In [441]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_NB,y_test)

0.3169761273209549

# Try Predicting with this shitty model

In [442]:
final_data[final_data['attribute_name']=='occasion'].iloc[18,:]

bc_product_id                                                        NaN
brand                                                   Emilia Wickstead
brand_canonical_url    https://www.modaoperandi.com/emilia-wickstead-...
brand_category                                      women:CLOTHING:PANTS
created_at                                 2019-11-07 19:40:24.663272+00
description            Emilia Wickstead' proves that her tailoring is...
details                Concealed zip fastening along back\nCompositio...
labels                                                                {}
mpn                                                               347393
name                          Francis Belted Checked Wool Wide-Leg Pants
notes                                                                NaN
product_id                                    01DS3P3NPXE2R3SFYNJNMCMSGZ
saleable                                                             NaN
tsv                    'belt':4A,49 'check':5A,34,3

In [443]:
#Take user input
description=input("Please Enter description")
Brand=input("Please Enter Brand")
category=input("Enter category")

Please Enter descriptionEmilia Wickstead' proves that her tailoring is just as covetable as the label's glamorous evening dresses. The label's 'Francis' pants are checked from checked virgin wool that's outlined in smart creases. Use the wide waist belt to temper the cuffed wide-leg hem
Please Enter Brandemi
Enter categorywomen:CLOTHING:PANTS


In [449]:
description=description.replace('\n',' ')
description=description.replace('/',' ')
description=description.replace(',',' ')
category=category.replace('\n',' ')
category=category.replace('/',' ')
category=category.replace(',',' ')

In [450]:
#Remove stopwords
nltk.download('stopwords')
columns=[description,category]
for j in range(0,len(columns)):
    corpus = []
    for i in range(0,1):
        new=re.sub('[^a-zA-Z]', ' ',columns[j])
        new = new.lower()
        new = new.split()
        ps = PorterStemmer()
        new = [ps.stem(word) for word in new if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        columns[j]=new
        

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [451]:
columns

['emilia wickstead prove tailor covet label glamor even dress label franci pant check check virgin wool outlin smart creas use wide waist belt temper cuf wide leg hem',
 'women cloth pant']

In [452]:
import scipy.sparse as sp
X=vect.transform(columns)

In [470]:
vect.transform(columns)

<2x987 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [475]:
vect.get_feature_names()

['ace',
 'acid',
 'acryl',
 'adler',
 'agn',
 'air',
 'airbrush',
 'airi',
 'airlift',
 'al',
 'alameda',
 'alex',
 'alexia',
 'alger',
 'alli',
 'alloisa',
 'alosoft',
 'alpaca',
 'ama',
 'amari',
 'amor',
 'andi',
 'andrea',
 'andrew',
 'angelica',
 'ankl',
 'ann',
 'anna',
 'annabel',
 'anorak',
 'ansel',
 'anson',
 'appliqu',
 'arabel',
 'arizona',
 'arliss',
 'arlo',
 'around',
 'ashlynn',
 'aslen',
 'asymmetr',
 'athena',
 'atla',
 'aubre',
 'aura',
 'aurora',
 'avelin',
 'avenu',
 'babi',
 'back',
 'backpack',
 'bag',
 'bailey',
 'balloon',
 'bambi',
 'banner',
 'barbara',
 'bardot',
 'bare',
 'bastian',
 'bb',
 'bea',
 'beachcomb',
 'beachwear',
 'bead',
 'bedford',
 'bell',
 'belt',
 'ben',
 'benelli',
 'bent',
 'beverli',
 'beya',
 'bi',
 'bike',
 'biker',
 'bindl',
 'birkenstock',
 'birkin',
 'bissett',
 'black',
 'blackcomb',
 'blake',
 'blanc',
 'blazer',
 'blend',
 'block',
 'blous',
 'blue',
 'boardwalk',
 'bodysuit',
 'boilersuit',
 'bomber',
 'bon',
 'bond',
 'boot',
 

In [458]:
y_pred_NB = classifier.predict(X.toarray())
y_pred_NB

ValueError: operands could not be broadcast together with shapes (2,987) (2837,) 

In [465]:
X_test.shape

(2262, 2837)

In [466]:
X_train.shape

(3393, 2837)

(3393,)